1. using the package management tool pip to install the necessary modules for running the application

In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -i https://pypi.tuna.tsinghua.edu.cn/simple opencv-python

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


2. import dependencies for exercise 1.1

In [1]:
import numpy as np
import cv2

3. Background subtraction 

First, create a VideoCapture object and read the frames from an input file. The first captured frame is regarded as the baseline image. 

Second, calculate the difference between the baseline and the gray frame.

Third, the difference frame named delta_frame is converted into a binary image which will be compared with a certain threshold (50), if it is greater than the threshold, it will be assigned the value of white (255), else it will be assigned the value of black (0).

Fourth, identify all the contours in the image and filter out small contours that do not belong to a car shape. Then draw green rectangles to highlight car shapes and draw red lines to indicate the main street area.

In [4]:
# create a VideoCapture object and read the frames from an input file
video = cv2.VideoCapture('Traffic_Laramie_1.mp4')

In [5]:
# check whether the video is opened successfully, if not, print error message
if (video.isOpened()== False): 
    print("Error opening video file")

In [6]:
# set the value of initial frame as None
initial_frame = None

In [7]:
# read through the video frame until video is completed or the user press 'q'
while True:
    check, frame = video.read()
    
    # breaks when reaching the end of the file 
    if frame is None:
        break

    # gray conversion: convert an image from one color space to another
    gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    
    # the first captured frame is regarded as the baseline image
    if initial_frame is None:
        initial_frame = gray_frame
        print(initial_frame.shape)
        continue

    # frame differencing: calculate the difference between the baseline and the gray frame
    delta_frame = cv2.absdiff(initial_frame,gray_frame)
    
    
    # the difference (the delta_frame) is converted into a binary image
    # if a particular pixel value is greater than a certain threshold (50)
    # if it is greater than the threshold, it will be assigned the value of white (255)
    # else it will be assigned the value of black (0)
    threshold_frame = cv2.threshold(delta_frame,50,255, cv2.THRESH_BINARY)[1]
    
    blur_frame = cv2.GaussianBlur(threshold_frame,(25,25),0)
    
    # identify all the contours in the image
    # this method has THREE parameters, (a) image, (b) contour retrieval mode and # (c) contour approximation method
    (contours,_) = cv2.findContours(blur_frame,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        # filter out any small contours that do not belong to a car shape
        if cv2.contourArea(c) < 6900:
            continue
        (x, y, w, h) = cv2.boundingRect(c)
        
        # draw green rectangles to highlight car shapes
        if y > 260:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 1)
            
        # draw the red line which indicates the main street area by cv2.line() method 
        # this is a horiontal line
        x1, y1 = 0, 260
        x2, y2 = 1040, 260
        
        # initialise the value of line thickness as 3
        line_thickness = 3
        
        # NOTE: color is in BGR, e.g., (0, 0, 255) means red
        cv2.line(frame, (x1, y1), (x2, y2), (0, 0, 255), thickness=line_thickness)
        
        # display the image
        cv2.imshow('Webcam', frame)
    
    # stop the program by pressing 'q'    
    if cv2.waitKey(1) == ord('q'):
        break

(600, 1040)


In [8]:
# after the loop above, release the video object
video.release()

# and destroy all windows
cv2.destroyAllWindows()

4. Improvements for Exercise 1.1 and 1.2

In the previous part, the first captured frame (initial frame) is regarded as the baseline image (i.e., background frame). In the method cv2.absdiff(), the difference between the baseline and the gray frame is calculated. Thus, I recommend using median frame as the background frame instead of initial frame in this part.

In [2]:
# define a function named median_frame with one parameter named path
def median_frame(path):
    
    # creage a VideoCapture object and read the frames from an input file with a given path
    video = cv2.VideoCapture(path)
    
    # generate 50 indexes based on 50 random frames for median
    indexes = video.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=50)
    
    # initialise an empty array to store frames
    frames = []
    
    # iterate each index and read video frame
    for i in indexes:
        video.set(cv2.CAP_PROP_POS_FRAMES, i)
        check, frame = video.read()
        frames.append(frame)
        
    # calculate the median of 50 random frames
    median_frame = np.median(frames, axis=0).astype(np.uint8)
    
    # the function returns the median frame
    return median_frame

The function backgroundDetection has the same function to the part 3 code. In other words, the function backgroundDetection can also detect and track moving cars in the video recording. In addition, the function backgroundDetection has its additional function, which is to return the value for number of cars go into city center. The value of num_car_into_city_center will be used for exercise 1.2 to calculate the value of cars per minute.

In [3]:
# define a function named backgroundDetection with three parameters
def backgroundDetection(path, numframes, rect_loc):
    
    # creage a VideoCapture object and read the frames from an input file with a given path
    video = cv2.VideoCapture(path)

    # get the height of the frame which detect cars at the main street only
    height = int(video.get(4))

    # get the median frame as the background frame based on the aforementioned function named median_frame
    background = median_frame(path)
    
    # convert the background frame in gray scale
    background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
    
    # initialise the count as the value of 0
    count = 0
    
    # initialise the numframes as the same value to parameter numframes
    numframes = numframes

    # initiaise the number of cars go into city center as the value of 0
    num_car_into_city_center = 0
    
    # create two empty arrays to store the x and y locations of the previous and current rectangles
    current_rect = []
    previous_rect = []
    
    # read through the video frame until video is completed or the user press 'q'
    while (video.isOpened()):
        check, frame = video.read()
        if check == True:
            count += 1
            
            # make a copy of the original frame
            original_frame = frame.copy()
            
            # convert frame to grayscale
            gray_scale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # checks whether frame count has reached a multiple of numframes
            if count % numframes == 0 or count == 1:
                difference_array = []
                
            # calculates absolute difference between gray scale and background
            abs_diff = cv2.absdiff(gray_scale, background)
            
            # converts the difference to binary using thresholding
            check, thres = cv2.threshold(abs_diff, 50, 255, cv2.THRESH_BINARY)
            
            # enlarge frame to increase contour detection
            enlarge_frame = cv2.dilate(thres, None, iterations=2)
            
            # append the final result into the array named difference_array
            difference_array.append(enlarge_frame)
            
            # check if difference_array has reached the required length
            
            if len(difference_array) == numframes:
                # calculate the sum the frames in the difference_array
                sum_num_frames = sum(difference_array)
                
                # save contours around the white enlarged areas
                contours, hierarchy = cv2.findContours(sum_num_frames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                # iterate every contour
                for i in range(len(contours)):
                    
                    # filter out any small contours that does not belong to a car shape
                    if cv2.contourArea(contours[i]) < 500:
                        continue
                    (x, y, w, h) = cv2.boundingRect(contours[i])


                    # draw green rectangles to highlight car shapes
                    if y > height / 2 and w * h > 6000:
                        cv2.rectangle(original_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                        
                        # append the locations of the rectangles to an array named current_rect
                        current_rect.append((x, y, w, h))

                # find the minimum length of the previous and current array of rectangles
                min_len = min(len(current_rect), len(previous_rect))
                for i in range(min_len):
                    if current_rect[i][0] > 400 and current_rect[i][0] < rect_loc and current_rect[i][1] > 350 and current_rect[i][1] < 400 and current_rect[i][0] < previous_rect[i][0]:
                        
                        # calculate the number of cars go into the city center
                        num_car_into_city_center += 1
                        
                        # print current number of cars go into the city center
                        print(num_car_into_city_center)

                # set the previous array of rectangles equal to the current array of rectangles
                previous_rect = current_rect
                
                # reset current array of rectangles
                current_rect = []

                # display the image
                cv2.imshow('Detected Objects', original_frame)
                
                # detect key event every 100ms , if the user press 'q', then stop the program 
                if cv2.waitKey(100) & 0xFF == ord('q'):
                    break
        else:
            break
            
            
    # after the loop above, release the video object
    video.release()
    
    # and destroy all windows
    cv2.destroyAllWindows() 
    
    # return the value for number of cars go into city center 
    return num_car_into_city_center

In [4]:
# get the value for numbers of cars go into city center on video 1 
num_car_into_city_center_1 = backgroundDetection('Traffic_Laramie_1.mp4', 4, 435)

# get the value for numbers of cars go into city center on video 2
num_car_into_city_center_2 = backgroundDetection('Traffic_Laramie_2.mp4', 5, 440)

1
2
3
4
5
6
1
2
3
4


References


arindomjit GitHub (2021). Motion_Detected_Alarm. Retrieved February 5, 2023,https://github.com/arindomjit/Motion_Detected_Alarm

Coursera. (n.d.).7.104 Exercise 15: Introduction to motion detection with OpenCV and Python. Retrieved February 5, 2023, from https://www.coursera.org/learn/uol-cm3065-intelligent-signal-processing/ungradedLab/0PXRU/7-104-exercise-15-introduction-to-motion-detection-with-opencv-and-python/lab?path=%2Fnotebooks%2FExercises%2FExercise%252015.%2520Introduction%2520to%2520motion%2520detection%2520with%2520OpenCV%2520and%2520Python.ipynb